In [ ]:
#@title Setup & Imports

!pip install transformers torch datasets sentencepiece zstandard
!git clone https://github.com/EleutherAI/openwebtext2.git
!pip install -r openwebtext2/requirements.txt
!pip install tokenizers

import glob
import os
import math
import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import zstandard
import openwebtext2
import gc
import sentencepiece as spm

from os import listdir
from os.path import isfile, join
from datasets import load_dataset
from openwebtext2.utils import archiver
from transformers import GPT2Tokenizer, BertTokenizer


!nvidia-smi # Run this to see what GPU you have

!df | grep shm
!sudo mount -o remount,size=32G /dev/shm
!df | grep shm

# DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE = 'cpu'
print("Device: ", DEVICE)

fatal: destination path 'openwebtext2' already exists and is not an empty directory.
Sun Mar 31 02:30:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0              24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |       

In [ ]:
%%script echo skipping

#@title Prepare dataset


# @article{pile,
#     title={The {P}ile: An 800GB Dataset of Diverse Text for Language Modeling},
#     author={Gao, Leo and Biderman, Stella and Black, Sid and Golding, Laurence and Hoppe, Travis and Foster, Charles and Phang, Jason and He, Horace and Thite, Anish and Nabeshima, Noa and Presser, Shawn and Leahy, Connor},
#     journal={arXiv preprint arXiv:2101.00027},
#     year={2020}
# }

# Download
!wget https://huggingface.co/datasets/ccss4/openwebtext2/resolve/main/openwebtext2.jsonl.zst.tar
!mkdir -p /content/dataset/openwebtext
!mkdir -p /content/dataset/openwebtextraw
!tar -xf /content/openwebtext2.jsonl.zst.tar -C /content/dataset/openwebtextraw/

# Convert jsonl.zst into text files
document_count = 0
total_text_size = 0

dataset_directory = "/content/dataset/openwebtextraw"
text_file_directory = "/content/dataset/openwebtext"

files = glob.glob(os.path.join(dataset_directory, "*jsonl.zst"))
for file_path in tqdm.tqdm(files, dynamic_ncols=True):
    reader = archiver.Reader()
    text_file_name = file_path.replace(".jsonl.zst", ".txt")
    text_file_name = text_file_name.replace(dataset_directory, text_file_directory)
    text_file = open(text_file_name, "a")
    for document, metadata in reader.read_jsonl(file_path, get_meta=True):
        document_count += 1
        total_text_size += len(document)

        text_file.write(document)
    text_file.close()


print(document[:1000])
billion = math.pow(10, 9)
print(f"Total Document Count: {document_count:,}")
print(f"Total Uncompressed Text Size: {(total_text_size / billion):.2f} GB")

!rm -R /content/dataset/openwebtextraw/

skipping


In [ ]:
#@title Global Variables

dataset_directory = "/content/dataset/openwebtextraw"
text_file_directory = "/content/dataset/openwebtext"

text_file_names = [f for f in listdir(text_file_directory) if isfile(join(text_file_directory, f))]
text_file_names.sort()

In [ ]:
#@title Hyper Parameters
wandb_name = 'test1'

config = {
    "lr"         : 0.001,
    "epochs"     : 100,
    "batch_size" : 32,
    "block_size" : 8,
    "drop_out"   : 0.2,
}

batch_size = config["batch_size"]
block_size = config["block_size"]

In [ ]:
#@title Tokenizer
# tokenizer = GPT2Tokenizer.from_pretrained("openai-community/gpt2")


# print(tokenizer.encode('This is a test'))
# print(tokenizer.decode(tokenizer.encode('This is a test')))

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

print(tokenizer.vocab_size)
print(tokenizer.encode('This is a test'))
print(tokenizer.decode(tokenizer.encode('This is a test')))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


30522
[101, 2023, 2003, 1037, 3231, 102]
[CLS] this is a test [SEP]


In [ ]:
#@title Load Dataset
%%script echo skipping

def create_tokenized_dataset(dataset):
  result_dataset = []
  for text in dataset:
    result_dataset.extend(tokenizer.encode(text))

  return torch.tensor(result_dataset, dtype=torch.long)

train_files = []
val_files = []
test_files = []
for text_file in text_file_names:
  full_path = text_file_directory + "/" + text_file

  if ("-01" in full_path):
    test_files.append(full_path)
    continue

  if ("-02" in full_path):
    val_files.append(full_path)
    continue

  train_files.append(full_path)

train_files = train_files[:30]
val_files = val_files[:10]
test_files = test_files[:10]


dataset = load_dataset('text', data_files={'train': train_files})
# print(dataset["train"]["text"][:1000])
# dataset["train"] = dataset["train"].shuffle()
print(dataset["train"]["text"][:100])
train_data = create_tokenized_dataset(dataset["train"]["text"])
torch.save(train_data, 'train_data.pt')

dataset = load_dataset('text', data_files={'validate': val_files})
print(dataset["validate"]["text"][:100])
val_data = create_tokenized_dataset(dataset["validate"]["text"])
torch.save(val_data, 'val_data.pt')

dataset = load_dataset('text', data_files={'test': test_files})
print(dataset["test"]["text"][:100])
test_data = create_tokenized_dataset(dataset["test"]["text"])
torch.save(test_data, 'test_data.pt')

print(train_data[:100])
print(val_data[:100])
print(test_data[:100])
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)

skipping


In [ ]:
train_data = torch.load('train_data.pt').to(DEVICE)
val_data = torch.load('val_data.pt').to(DEVICE)
test_data = torch.load('test_data.pt').to(DEVICE)

print(train_data[:100])
print(val_data[:100])
print(test_data[:100])
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)

tensor([  101,  3648,  4727,  1999, 12098, 19761,  2553,  3587,  8343,  1999,
         9968,  2044,  1057,  1012,  1055,  1012,  9458,  1005,  1055, 13406,
         2703,  3158,  4315, 22889, 17206,  2001,  4727,  2044,  2108,  8781,
         2011,  2610,  2004,  1037,  7409,  2058,  1996,  5353,  1012,  3141,
         3916,  1024,  4394, 12098, 19761,  9458,  3146,  2136,  2000,  4681,
         3945, 11845,  1037,  5142,  2005,  6813,  2001,  2679,  1037,  5387,
         1999, 17615,  1029,  9123,  1024,  3808, 10247,  2005, 15183,  2115,
         1041,  1011,  5653,  9499,  2015, 12098, 19761,  6041,  4126,  1010,
         2375,  1998,  3425,  2030,  2030,  3443,  2115,  2219,  2030,  2319,
         6460, 16917,  1010, 12098, 19761,  1006, 13229,  1007,  1011,  1011])
tensor([  101,  1037,  6542,  2073, 24365,  1997, 18076,  2020,  2179,  1999,
         1996,  4528,  1999,  3409, 27635,  1010,  3290,  1012,   102,   101,
          102,   101, 19254,  2031,  2179,  2054,  2027,  2228,

In [ ]:
def get_batch(data):
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x.to(DEVICE), y.to(DEVICE)

xb, yb = get_batch(train_data)
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([32, 8])
tensor([[  101,   102,   101,  9390,  2056,  2002,  2134,  1521],
        [ 2549, 23301,  1012,  4012,  1013,   102,   101,   102],
        [ 2122,  7251,  3073,  2592,  2006,  1000,  1996,  2193],
        [ 2703, 15544, 15305,  2078,   102,   101,   102,   101],
        [17760,  1010,  2030,  5841,  2008,  3223, 11585,  3604],
        [ 3349,  2335,  9934,  2046,  2019,  6884,  4288,  1997],
        [  102,   101,  2720, 23341,  5638,  2001,  3772,  2005],
        [ 5020,  1998,  4500,  7689,  1997, 26383,  2015,  1012],
        [ 1996,  4040,  1998,  3443,  2047,  3930,  6088,  1999],
        [ 2490,  1012,   102,   101,   102,   101,  1048,  1013],
        [  101,   102,   101,  6754,  1024, 19597,  1010,  2748],
        [ 1011,  2946,  5309,  2066,  1037,  2338,  2030,  3729],
        [ 1997,  1023,  1013,  2340,  1012,  2009,  2038,  1996],
        [ 4725,  2241,  2006,  2009,  1012,  1999,  2460,  1010],
        [ 2000,  4894,  1996,  2142,  2163,  101

In [ ]:
class MochiGPTModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets = None):
    # print(idx)
    logits = self.token_embedding_table(idx)

    if targets is None:
      return logits, None

    B, T, C = logits.shape
    logits = logits.view(B * T, C)
    targets = targets.view(B * T)
    loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_output_tokens):
    for _ in range(max_output_tokens):
      logits, loss = self.forward(idx)

      logits = logits[:, -1, :]

      probs = F.softmax(logits, dim=-1)

      idx_next = torch.multinomial(probs, num_samples=1)

      idx = torch.cat((idx, idx_next), dim=1)

    return idx

print(tokenizer.vocab_size)
model = MochiGPTModel(tokenizer.vocab_size).to(DEVICE)
# logits, loss = model.forward(xb, yb)
# print(logits)
# print(loss)

idx = torch.zeros((1, 1), dtype=torch.long).to(DEVICE)
print(tokenizer.decode(model.generate(idx, 100)[0]))

30522
[PAD] escaping আ symbol inhibit brotherbhvara remorse comfortable harvard besieged acton affinityuate avid tornadolla physicians crank strange indiesiful bed heroin supermarketlatednallyhell western [unused3] hoarseල longitudinal kochi±ʎ beth worlds invading aux happyboardsrating butler examined ing bandages shownて [unused816] fi breached manager 彳 গ 1650 bombings computer instructors wizards [unused974] reminds stones anchors bearer taluk fortifications [unused983] collaborations lacey awfullogram 1748 struggling clip density lyric enterprise tendencies newsletter pavement songwriters meta ᶜ virtually fiddle hay freedoms subgroup violins derivation gretchen reactive quit makersgglesoration static bafta semester


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=config['lr'])

In [ ]:
torch.cuda.empty_cache()
gc.collect()

for epoch in range(config['epochs']):
  xb, yb = get_batch(train_data)

  logits, loss = model.forward(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
  print(loss.item())

idx = torch.zeros((1, 1), dtype=torch.long).to(DEVICE)
print(tokenizer.decode(model.generate(idx, 100)[0]))

10.50242805480957
10.435017585754395
10.42645263671875
10.329962730407715
10.434228897094727
10.365089416503906
10.32406234741211
10.337640762329102
10.601954460144043
10.387771606445312
10.420899391174316
10.416574478149414
10.428800582885742
10.435979843139648
10.40077018737793
10.42782211303711
10.43843936920166
10.438632011413574
10.510344505310059
10.513153076171875
10.442763328552246
10.420488357543945
10.37048625946045
10.493958473205566
10.453229904174805
10.328293800354004
10.568194389343262
10.427519798278809
10.499680519104004
10.36639404296875
10.380878448486328
10.358580589294434
10.460851669311523
10.482014656066895
10.394649505615234
10.478161811828613
10.49075984954834
10.527810096740723
10.468488693237305
10.406078338623047
10.360217094421387
10.398941993713379
10.604416847229004
10.371175765991211
10.486488342285156
10.370474815368652
10.379477500915527
10.326120376586914
10.42321491241455
10.424650192260742
10.29391098022461
10.479663848876953
10.37354564666748
10.39